The website we scraped its data for this project: https://carsheet.io/

In [1]:
!pip install requests beautifulsoup4 pandas openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# the URL and an empty list to store data
base_url = "https://carsheet.io/aston-martin,audi,bentley,bmw,ferrari,ford,mercedes-benz,nissan/2023,2024/"

data = []

# Function to extract data from a single page
def scrape_page(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract table headers
        headers = [th.text.strip() for th in soup.find_all('table', {'id': 'carsheet'})[0].find_all('th')]
        
        # Extract table rows
        rows = soup.find('table', {'id': 'carsheet'}).find('tbody').find_all('tr')  # Get rows from <tbody>
        
        for row in rows:
            cols = row.find_all('td')
            if len(cols) == len(headers):  # Ensure row length matches header length
                car_info = {headers[i]: cols[i].text.strip() for i in range(len(headers))}
                data.append(car_info)
                
    except requests.RequestException as e:
        print(f"An error occurred: {e}")

# Generate page ranges dynamically
page_ranges = [f"{i}-{i+50}" for i in range(0, 2000, 50)]

# Scrape all pages
for page_range in page_ranges:
    page_url = f"{base_url}{page_range}/"
    scrape_page(page_url)
    

# Create a DataFrame and save to Excel
df = pd.DataFrame(data)
df = df.drop_duplicates()
df.to_excel('cars_data.xlsx', index=False)

In [22]:
pd.DataFrame(data).head()

,Make,Model,Year,Trim,MSRP,Invoice Price,Used/New Price,Body Size,Body Style,Cylinders,Engine Aspiration,Drivetrain,Transmission,Horsepower,Torque,Highway Fuel Economy
0,Aston Martin,DBX707,2024,Base,"$242,000",,"$242,000",Large,SUV,V8,Twin-Turbo,AWD,automatic,697 hp @ 6000 rpm,663 ft-lbs. @ 2750 rpm,20 mpg
1,Audi,A3,2024,Premium w/40 TFSI,"$35,800","$33,653","$35,800",Compact,Sedan,I4,Turbocharged,FWD,automatic,201 hp @ 4800 rpm,221 ft-lbs. @ 4100 rpm,37 mpg
2,Audi,A3,2024,Premium w/40 TFSI,"$37,800","$35,533","$37,800",Compact,Sedan,I4,Turbocharged,AWD,automatic,201 hp @ 5000 rpm,221 ft-lbs. @ 4000 rpm,34 mpg
3,Audi,A3,2024,Premium Plus w/40 TFSI,"$41,400","$38,917","$41,400",Compact,Sedan,I4,Turbocharged,AWD,automatic,201 hp @ 5000 rpm,221 ft-lbs. @ 4000 rpm,34 mpg
4,Audi,A3,2024,Premium Plus w/40 TFSI,"$39,400","$37,037","$39,400",Compact,Sedan,I4,Turbocharged,FWD,automatic,201 hp @ 4800 rpm,221 ft-lbs. @ 4100 rpm,37 mpg


In [23]:
df['Make'].unique()

array(['Aston Martin', 'Audi', 'BMW', 'Bentley', 'Ford', 'Mercedes-Benz',
       'Nissan'], dtype=object)